Dataset is downloaded from https://github.com/chandrikadeb7/Face-Mask-Detection

# IMPORT

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import math
import os
import shutil
import sys

from tensorflow.keras.callbacks import CSVLogger
from tensorflow.keras import datasets
from utils.model import AugLayer, VGG, EfficientNetB0, CheckpointCallback, CustomLRDecay, plot_history

from tensorflow.keras.applications.efficientnet import preprocess_input

from tensorflow.keras import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Activation, Flatten, Add
from tensorflow.keras.layers import Conv2D, MaxPool2D, AveragePooling2D, ZeroPadding2D, GlobalAveragePooling2D
from tensorflow.keras.layers import BatchNormalization, Dropout
import cv2

import matplotlib.pyplot as plt

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import tensorflow as tf
from PIL import Image
from sklearn.model_selection import train_test_split

%matplotlib inline

# GET DATASET

In [4]:
PATH_DATASET = 'drive/My Drive/Colab Notebooks/face-mask-recognition/dataset/'
PATH_DATASET_NEG = 'drive/My Drive/Colab Notebooks/face-mask-recognition/dataset/0/'
PATH_DATASET_POS = 'drive/My Drive/Colab Notebooks/face-mask-recognition/dataset/1/'
MIN_PIXEL = 32
IMAGE_SIZE = (112, 112)

In [5]:
# images = []
# label = []
# for name in os.listdir(PATH_DATASET_POS):
#   img = Image.open(PATH_DATASET_POS+name)
#   if (img.size[0] >= MIN_PIXEL) and (img.size[1] >= MIN_PIXEL):
#     img = img.resize(IMAGE_SIZE, Image.LANCZOS)
#     img = np.asarray(img)
#     if img.shape == (*IMAGE_SIZE, 3):
#       images.append(img)
#       label.append(1)
#     else: print(img.shape)
#   else: print(img.size)

# for name in os.listdir(PATH_DATASET_NEG):
#   img = Image.open(PATH_DATASET_NEG+name)
#   if (img.size[0] >= MIN_PIXEL) and (img.size[1] >= MIN_PIXEL):
#     img = img.resize(IMAGE_SIZE, Image.LANCZOS)
#     img = np.asarray(img)
#     if img.shape == (*IMAGE_SIZE, 3):
#       images.append(img)
#       label.append(0)
#     else: print(img.shape)
#   else: print(img.size)

# images = np.array(images)
# label = np.array(label)

# np.save(PATH_DATASET+'images.npy', images)
# np.save(PATH_DATASET+'label.npy', label)

In [6]:
images = np.load(PATH_DATASET+'images.npy')
label = np.load(PATH_DATASET+'label.npy')

In [7]:
# https://datascience.stackexchange.com/questions/15135/train-test-validation-set-splitting-in-sklearn
X_train, X_test, y_train, y_test = train_test_split(images, label, test_size=0.2, stratify=label, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, stratify=y_train, random_state=42) # 0.25 x 0.8 = 0.2

# TRAIN MODEL

In [8]:
BATCH_SIZE = 512
EPOCHS = 30
PATH_CHECKPOINT = './face-mask-recognition/checkpoint'
PATH_CSV = './face-mask-recognition/training.log'
LR_PROFILE = [0.1, 0.01, 0.001]
LR_EPOCHS = [10, 20, EPOCHS]

AUG_RESIZE= None # (128,128)
AUG_CONTRAST = 0.10
AUG_CROP = None
AUG_FLIP = 'horizontal'
AUG_ROT = 30.
AUG_TRANSLATION = (0.10, 0.10)
AUG_ZOOM = 0.10

shutil.rmtree(PATH_CHECKPOINT, ignore_errors=True)
os.makedirs(PATH_CHECKPOINT)

In [9]:
aug_layer = AugLayer(AUG_RESIZE, None, AUG_CONTRAST,
                     AUG_CROP, AUG_FLIP, AUG_ROT,
                     AUG_TRANSLATION, AUG_ZOOM)

model = EfficientNetB0((*IMAGE_SIZE,3), 1, aug_layer, "imagenet", False)

model.compile(optimizer=tf.keras.optimizers.SGD(momentum=0.9, nesterov=True), loss='binary_crossentropy', metrics=['acc'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
augmentation_layer (Sequenti (None, 112, 112, 3)       0         
_________________________________________________________________
efficientnetb0 (Functional)  (None, None, None, 1280)  4049571   
_________________________________________________________________
global_avg_pool (GlobalAvera (None, 1280)              0         
_________________________________________________________________
dropout_layer1 (Dropout)     (None, 1280)              0         
_________________________________________________________________
fc_layer (Dense)             (None, 1280)              1639680   
_________________________________________________________________
dropout_layer2 (Dropout)     (None, 1280)              0         
_________________________________________________________________
output_layer (Dense)         (None, 1)                 1

In [10]:
callbacks = [#CheckpointCallback(PATH_CHECKPOINT),
             CustomLRDecay(LR_PROFILE, LR_EPOCHS),
                          CSVLogger(PATH_CSV)]

history = model.fit(X_train, y_train,
                    steps_per_epoch = math.ceil(X_train.shape[0] / BATCH_SIZE),
                    validation_data=(X_val, y_val),
                    validation_steps = math.ceil(X_test.shape[0] / BATCH_SIZE),
                    epochs=EPOCHS,
                    callbacks=callbacks,
                    workers=4)

Epoch 1/30

Epoch 00001: LearningRateScheduler reducing learning rate to 0.1.
5/5 [==============================] - 16s 2s/step - loss: 0.5413 - acc: 0.6970 - val_loss: 0.2093 - val_acc: 0.9203
Epoch 2/30

Epoch 00002: LearningRateScheduler reducing learning rate to 0.1.
5/5 [==============================] - 3s 530ms/step - loss: 0.1476 - acc: 0.9549 - val_loss: 0.1499 - val_acc: 0.9281
Epoch 3/30

Epoch 00003: LearningRateScheduler reducing learning rate to 0.1.
5/5 [==============================] - 3s 530ms/step - loss: 0.0629 - acc: 0.9814 - val_loss: 0.1202 - val_acc: 0.9438
Epoch 4/30

Epoch 00004: LearningRateScheduler reducing learning rate to 0.1.
5/5 [==============================] - 3s 529ms/step - loss: 0.0506 - acc: 0.9842 - val_loss: 0.0763 - val_acc: 0.9686
Epoch 5/30

Epoch 00005: LearningRateScheduler reducing learning rate to 0.1.
5/5 [==============================] - 3s 529ms/step - loss: 0.0319 - acc: 0.9907 - val_loss: 0.0522 - val_acc: 0.9817
Epoch 6/30

Epoch

# EVALUATE MODEL

In [11]:
model.evaluate(X_test, y_test, batch_size=512)

2/2 [==============================] - 2s 801ms/step - loss: 0.0406 - acc: 0.9922


[0.04062690585851669, 0.9921568632125854]